In [32]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

In [33]:
catalog_lunar_dir = "../data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv"

catalog_lunar = pd.read_csv(catalog_lunar_dir)

catalog_lunar

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq
...,...,...,...,...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156,1974-10-14T17:43:00.000000,63780.0,evid00156,impact_mq
72,xa.s12.00.mhz.1975-04-12HR00_evid00191,1975-04-12T18:15:00.000000,65700.0,evid00191,impact_mq
73,xa.s12.00.mhz.1975-05-04HR00_evid00192,1975-05-04T10:05:00.000000,36300.0,evid00192,impact_mq
74,xa.s12.00.mhz.1975-06-24HR00_evid00196,1975-06-24T16:03:00.000000,57780.0,evid00196,impact_mq


In [34]:
catalog_lunar.columns
row = catalog_lunar.iloc[6]
arrival_time = datetime.strptime(row["time_abs(%Y-%m-%dT%H:%M:%S.%f)"], '%Y-%m-%dT%H:%M:%S.%f')
arrival_time_relative = row["time_rel(sec)"]
test_filename = row.filename
data_dir = "../data/lunar/training/data/S12_GradeA/"
csv_file = f'{data_dir}{test_filename}.csv'
raw_data = pd.read_csv(csv_file)
raw_data

,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),velocity(m/s)
0,1970-06-26T00:00:00.116000,0.000000,-6.727977e-16
1,1970-06-26T00:00:00.266943,0.150943,-8.646711e-16
2,1970-06-26T00:00:00.417887,0.301887,-9.298738e-16
3,1970-06-26T00:00:00.568830,0.452830,-8.589095e-16
4,1970-06-26T00:00:00.719774,0.603774,-7.139047e-16
...,...,...,...
572418,1970-06-27T00:00:02.832981,86402.716981,5.039820e-17
572419,1970-06-27T00:00:02.983925,86402.867925,-9.191068e-18
572420,1970-06-27T00:00:03.134868,86403.018868,-2.796955e-17
572421,1970-06-27T00:00:03.285811,86403.169811,-9.037156e-17


In [88]:
chunk_size = 10000
total_rows = raw_data.shape[0]
total_rows

results_df = pd.DataFrame(columns=["chunk", "label"])
results_df

,chunk,label


In [89]:
for start in range(0, total_rows, chunk_size):
    end = start + chunk_size

    chunk = raw_data.iloc[start:end]
    # print(chunk)

    data_df = pd.DataFrame(chunk["time_rel(sec)"])
   
    if arrival_time_relative < data_df["time_rel(sec)"].values.max() and arrival_time_relative > data_df["time_rel(sec)"].values.min():
        new_row = pd.DataFrame({"chunk": [chunk], "label": [1]})
        # print(new_row)
        results_df = pd.concat([results_df, new_row], ignore_index=True)
    else:
        new_row = pd.DataFrame({"chunk": [chunk], "label": [0]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)




results_df

,chunk,label
0,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel(...,0
1,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0
2,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0
3,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0
4,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0
5,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0
6,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0
7,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0
8,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0
9,time_abs(%Y-%m-%dT%H:%M:%S.%f) time_rel...,0


In [91]:
label_counts = results_df["label"].value_counts()

count_label_1 = label_counts.get(0, 1)

count_label_1

57